In [1]:
from gurobipy import *

In [2]:
%pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
# initialize the data for the problem

no_days = 5
no_airports = 3

data = [ [100,200,100,400,300],
        [50,50,50,50,50],
        [25,25,25,25,25],
        [25,25,25,25,25],
        [40,40,40,40,40],
        [400,200,300,200,400]
    ]

reposition_costs = [[[0,7,3],
                     [7,0,6],
                     [3,6,0]]]*5
print(reposition_costs)
warehouse_costs = 10


new_cargoes = [ [[ 0 for j in range( no_airports ) ] for i in range( no_airports ) ] for k in range( no_days ) ]

                
new_cargoes = [ 
    [ [0, 100, 50], [25, 0, 25], [40, 400, 0] ], #Monday - Day 0
    [ [0, 200, 50], [25, 0 ,25], [40, 200, 0] ], #Tuesday - Day 1
    [ [0, 100, 50], [25, 0 ,25], [40, 300, 0] ], #Wednesday - Day 2
    [ [0, 400, 50], [25, 0 ,25], [40, 200, 0] ],  #Thursday - Day 3
    [ [0, 300, 50], [25, 0 ,25], [40, 400, 0] ],  #Friday - Day 4
]



[[[0, 7, 3], [7, 0, 6], [3, 6, 0]], [[0, 7, 3], [7, 0, 6], [3, 6, 0]], [[0, 7, 3], [7, 0, 6], [3, 6, 0]], [[0, 7, 3], [7, 0, 6], [3, 6, 0]], [[0, 7, 3], [7, 0, 6], [3, 6, 0]]]


Airport 1 is A, Airport 2 is B, Airport 3 is C

Monday is day 1, Tuesday is day 2, Wednesday is day 3, Thursday is day 4, Friday is day 5

In [4]:
# create an empty model 
myModel = Model( "project" ) 

# create decision variables

# number of aircrafts repositioned from airport i to airport j on day t
reposition = [ [[ 0 for j in range( no_airports ) ] for i in range( no_airports ) ] for k in range( no_days ) ] 
for k in range( no_days ): 
    for i in range( no_airports ):
        for j in range ( no_airports ):
            curVar = myModel.addVar( vtype = GRB.INTEGER , name = "repositioning_day" + str(k+1) + "_from" + str(i+1) + "_to" + str(j+1) )
            reposition[ k ][ i ][ j ] = curVar

# number of cargoes shipped from airport i to j on day t
ship = [ [[ 0 for j in range( no_airports ) ] for i in range( no_airports ) ] for k in range( no_days ) ] 
for k in range( no_days ): 
    for i in range( no_airports ):
        for j in range ( no_airports ):
            curVar = myModel.addVar( vtype = GRB.INTEGER , name = "shipment_on_day" + str(k+1) + "_from" + str(i+1) + "_to" + str(j+1) )
            ship[ k ][ i ][ j ] = curVar
            
             
# number of cargoes that have to be shipped from airport i to airport j on day t
x = [ [[ 0 for j in range( no_airports ) ] for i in range( no_airports ) ] for k in range( no_days ) ] 
for k in range( no_days ): 
    for i in range( no_airports ):
        for j in range ( no_airports ):
            curVar = myModel.addVar( vtype = GRB.INTEGER , name = "cargoes_needed_to_be_shipped_day" + str(k+1) + "_from" + str(i+1) + "_to" + str(j+1) )
            x[ k ][ i ][ j ] = curVar

# number of aircrafts available at airport i on day t
# availability = [ [ 0 for i in range( no_airports ) ] for k in range( no_days ) ] 
# for k in range( no_days ): 
#     for i in range( no_airports ):
#             curVar = myModel.addVar( vtype = GRB.INTEGER , name = "available_day" + str(k+1) + "_airport" + str(i+1))
#             availability[ k ][ i ] = curVar

# register all decision variables into model 
myModel.update()

Restricted license - for non-production use only - expires 2024-10-28


In [5]:
# create objective function 
objExpr = LinExpr() 

for k in range( no_days ):
    for i in range( no_airports ): 
        for j in range( no_airports ): 
            if i != j:
                if k == 4:
                    objExpr += reposition[ k ][ i ][ j ] * reposition_costs[ k ][ i ][ j ] + (x[ k ][ i ][ j ] - ship[ k ][ i ][ j ]) * warehouse_costs * 3
                else:
                    objExpr += reposition[ k ][ i ][ j ] * reposition_costs[ k ][ i ][ j ] + (x[ k ][ i ][ j ] - ship[ k ][ i ][ j ]) * warehouse_costs
        
myModel.setObjective( objExpr , GRB.MINIMIZE )

In [6]:
# create constraints that ensures there are 1200 aircrafts running at any given day
for k in range(no_days):
    constExpr = LinExpr() 
    for i in range(no_airports):
        for j in range(no_airports):
            if i != j :
                constExpr += ship[ k ][ i ][ j ] + reposition[ k ][ i ][ j ]
            else:
                constExpr += reposition[ k ][ i ][ j ]
    myModel.addConstr( lhs = constExpr , sense = GRB.EQUAL , rhs = 1200 , name = "aircraftno_day" + str(k+1))

# constraint that ensures the number of aircrafts leaving an airport or being held on the ground
# on any given day is the same as the number of aircrafts coming into the airport or being held 
# on the ground the previous day
for k in range(no_days):
    for i in range(no_airports):
        constExpr = LinExpr()
        for j in range(no_airports):
            if i != j :
                if k == 0 :
                    constExpr += ship[ k ][ i ][ j ] - ship[ 4 ][ j ][ i ]
                    constExpr+= reposition[ k ][ i ][ j ] - reposition[ 4 ][ j ][ i ]
                else:
                    constExpr += ship[ k ][ i ][ j ] - ship[ k-1 ][ j ][ i ]
                    constExpr+= reposition[ k ][ i ][ j ] - reposition[ k-1 ][ j ][ i ]
            else:
                if k == 0 :
                    constExpr+= reposition[ k ][ i ][ j ] - reposition[ 4 ][ j ][ i ]
                else:
                    constExpr+= reposition[ k ][ i ][ j ] - reposition[ k-1 ][ j ][ i ]
        myModel.addConstr( lhs = constExpr , sense = GRB.EQUAL , rhs = 0 , name = "aircraftflow_airport_" + str(i+1) + "_day_"+str(k+1))
    

# create constraint that ensure the cargoes that need to be shipped from airport i to j on day t 
# is the same as the cargoes that were available the previous day, minus the cargoes that were shipped
# and put in the warehouse the previous days, plus the new cargoes 
for k in range(no_days):
    for i in range(no_airports):
        for j in range(no_airports):
            constExpr = LinExpr()
            if i != j :
                if k == 0 :
                    constExpr += x[ k ][ i ][ j ] - x[ 4 ][ i ][ j ]
                    constExpr += ship[ 4 ][ i ][ j ] - new_cargoes[ k ][ i ][ j ]
                else:
                    constExpr += x[ k ][ i ][ j ] - x[ k-1 ][ i ][ j ]
                    constExpr += ship[ k-1 ][ i ][ j ] - new_cargoes[ k ][ i ][ j ]
            myModel.addConstr( lhs = constExpr , sense = GRB.EQUAL , rhs = 0 , name = "cargoes_needed_to_be_carried_from_" + str(i+1) + "_to_"+str(j+1)+"_day_"+str(k+1))

# create constaint that ensures number of cargoes shipped on day t from i to j is less or equal to 
# the availble cargoes
for k in range(no_days):
    for i in range(no_airports):
        for j in range(no_airports):
            constExpr = LinExpr()
            if i != j :
                constExpr += x[ k ][ i ][ j ] - ship[ k ][ i ][ j ]
            myModel.addConstr( lhs = constExpr , sense = GRB.GREATER_EQUAL , rhs = 0 , name = "ship/availble_airport" + str(i+1) +"to"+ str(j+1)+"day"+str(k+1))
        
#register objective and constraints into model
myModel.update()

In [7]:
#check the status of the model:
curStatus = myModel.status
if curStatus in (GRB.Status.INF_OR_UNBD, GRB.Status.INFEASIBLE, GRB.Status.UNBOUNDED):
    print("Could not find the optimal solution")
    exit(1)
    
# write model in a file, find optimal solution, print optimal objective value and solution
myModel.write( filename = "homework9.lp" ) 
myModel.optimize() 
print( "optimal objective value " + str( myModel.ObjVal ) ) 
print( "optimal solution" ) 
allVars = myModel.getVars()
#for curVar in allVars:
#    print( curVar.varName + " : " + str( curVar.x ) ) 

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i3-1125G4 @ 2.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 135 columns and 375 nonzeros
Model fingerprint: 0xbd81c98e
Variable types: 0 continuous, 135 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+03]
Presolve removed 60 rows and 54 columns
Presolve time: 0.00s
Presolved: 50 rows, 81 columns, 315 nonzeros
Variable types: 0 continuous, 81 integer (0 binary)

Root relaxation: objective 2.172500e+04, 37 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    21725.000000 21725.0000  0.00%   

In [8]:
print( "optimal objective value ", str( myModel.ObjVal ), '\n' ) 
myModel.printAttr("X")

optimal objective value  21725.0 


    Variable            X 
-------------------------
repositioning_day1_from2_to1          275 
repositioning_day1_from2_to2           20 
repositioning_day1_from2_to3          165 
repositioning_day2_from2_to1           85 
repositioning_day2_from2_to2          220 
repositioning_day2_from2_to3          265 
repositioning_day3_from2_to1          495 
repositioning_day3_from2_to3          165 
repositioning_day4_from1_to1          110 
repositioning_day4_from2_to1          170 
repositioning_day4_from2_to3          180 
repositioning_day5_from2_to3          550 
shipment_on_day1_from1_to2           15 
shipment_on_day1_from1_to3           50 
shipment_on_day1_from2_to1           25 
shipment_on_day1_from2_to3           25 
shipment_on_day1_from3_to1           40 
shipment_on_day1_from3_to2          585 
shipment_on_day2_from1_to2          290 
shipment_on_day2_from1_to3           50 
shipment_on_day2_from2_to1           25 
shipment_on_day2_from2_to3